In [1]:
import os
import re
import json
import gzip
import time
import zipfile
import requests
import numpy as np
import pandas as pd
from datetime import date
from decimal import Decimal
CACHE = {} # глобальный кэш, в процессе скрейпинга не обновлять!
SETS = './sets/' # папка с файлами с наборами

In [2]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_cache(filename='cache.gz'): # загрузка кэша
    global CACHE
    if os.path.exists(filename) and len(CACHE)==0:
        with gzip.open(filename, 'rb') as gzip_ref:
            CACHE = json.load(gzip_ref)
        print(f'Загружено {len(CACHE)} записей из {filename} в кэш')


def save_cache(filename='cache.gz'): # сохранение кэша
    global CACHE
    with gzip.open(filename, 'wb') as gzip_file:
        json_data = json.dumps(CACHE, ensure_ascii=False, default=default_serializer)
        gzip_file.write(json_data.encode('utf-8'))
    print(f'Сохранено {len(CACHE)} записей в {filename}')
    
    
def load_gzon(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return json.load(gzip_ref)
    else:
        return []
    

def save_gzon(filename):
    data = globals().get(filename)
    if data is not None:
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(json_data)} записей в {filename}.gz')
    else:
        print(f'Переменная {filename} не найдена')
        
     
def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')

In [4]:
ecology[ecology['start'].str.len() < 1]

,start,end,air,mean,noise,dirt,digging,capital,house_lines,clinics,roads,parks,transports,city_places,live_area,streets,trashes,factories,markets,autostations,gasstations,bad_gasstations,buildgrounds
185065,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.582339408, 55.751895783], 'type': 'Point'}"
185066,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.580817726, 55.570154208], 'type': 'Point'}"
185068,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.500756262, 55.801785039], 'type': 'Point'}"
185070,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.596979823, 55.715937227], 'type': 'Point'}"
185076,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.48556881, 55.786097706], 'type': 'Point'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185821,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.400555311, 55.643330548], 'type': 'Point'}"
185822,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.65791113, 55.637167012], 'type': 'Point'}"
185823,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.735652297, 55.776123852], 'type': 'Point'}"
185824,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.564242096, 55.663143468], 'type': 'Point'}"


In [5]:
buildgrounds = load_dataset('2941')

In [6]:
buildgrounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   global_id             771 non-null    int64  
 1   ID                    771 non-null    int64  
 2   ObjectName            771 non-null    object 
 3   AdmArea               771 non-null    object 
 4   District              771 non-null    object 
 5   ObjectAddress         771 non-null    object 
 6   Coordinates           771 non-null    object 
 7   ConsrtuctionWorkType  771 non-null    object 
 8   MainFunctional        771 non-null    object 
 9   SourceOfFinance       771 non-null    object 
 10  DevelopmentArea       751 non-null    float64
 11  FloorsAmount          771 non-null    object 
 12  ObjectStatus          771 non-null    object 
 13  DeveloperInfo         771 non-null    object 
 14  CadastralNumber       771 non-null    object 
 15  AreaCoordinates       7

In [7]:
buildgrounds.head()

,global_id,ID,ObjectName,AdmArea,District,ObjectAddress,Coordinates,ConsrtuctionWorkType,MainFunctional,SourceOfFinance,DevelopmentArea,FloorsAmount,ObjectStatus,DeveloperInfo,CadastralNumber,AreaCoordinates,GPZUDocumentNumber,GPZUDocumentDate,GPZUDocumentStatus,Permission,EntryState,EntryAddReason,EntryModifyReason,EntryDeleteReason,ParentEntries,ChildEntries,FunctionalForNG,geoData
0,272726564,771,"Телевизионный комплекс ОАО ""Телекомпания НТВ""",Северо-Восточный административный округ,Останкинский район,"Москва, СВАО, район Останкинский, Новомосковская улица, вл. 18-22","37.619060188133,55.817768093242",новое строительство,"Объект производства электронных компонентов, аппаратур для радио, телевидения и связи",внебюджетные источники,6545.00,8,в строительстве,"[{'is_deleted': 0, 'global_id': 64, 'OrgForm': 'Акционерное общество', 'OrgName': 'Телекомпания НТВ'}]",77:02:0022005:1,,RU77-174000-001921,08.09.2010,действует,"[{'is_deleted': 0, 'global_id': 6422, 'PermissionNumber': 'RU77174000-006932', 'PermissionBeginDate': '23.05.2012', 'PermissionEndDate': '30.09.2024', 'PermissionStatus': 'действует', 'PermissionFile': [{'is_deleted': 0, 'global_id': 43255, 'PermissionFile': '9343f469-01c8-4975-841f-4d41b9b0d808'}]}, {'is_deleted': 0, 'global_id': 42554, 'PermissionNumber': 'RU77129000-007035', 'PermissionBeginDate': '06.07.2012', 'PermissionEndDate': '30.06.2016', 'PermissionStatus': 'срок действия истек', 'PermissionFile': []}, {'is_deleted': 0, 'global_id': 42557, 'PermissionNumber': 'RU77129000-001232', 'PermissionBeginDate': '16.10.2007', 'PermissionEndDate': '14.04.2011', 'PermissionStatus': 'срок действия истек', 'PermissionFile': []}, {'is_deleted': 0, 'global_id': 42559, 'PermissionNumber': '14804', 'PermissionBeginDate': '26.12.2006', 'PermissionEndDate': '01.01.2009', 'PermissionStatus': 'срок действия истек', 'PermissionFile': []}]",активна,новый объект,,None,[],[],Промышленно-производственный объект,"{'coordinates': [37.619060188, 55.817768093], 'type': 'Point'}"
1,272726691,738,Одноквартирные жилые дома (3-й пусковой комплекс),Северо-Западный административный округ,район Куркино,"Москва, СЗАО, район Куркино, мкр. 1 АБВ, корп. 44, 46, 49, 51, 53, 55","37.379301300613,55.906705621953",новое строительство,Малоэтажный многоквартирный дом,городской бюджет,230.20,2,в строительстве,"[{'is_deleted': 0, 'global_id': 125, 'OrgForm': 'Акционерное общество', 'OrgName': 'УЭЗ'}]",77:08:0001007:1,,RU77-146000-005490,16.03.2012,аннулирован,"[{'is_deleted': 0, 'global_id': 15218, 'PermissionNumber': 'RU77146000-007713', 'PermissionBeginDate': '07.02.2013', 'PermissionEndDate': '30.11.2013', 'PermissionStatus': 'срок действия истек', 'PermissionFile': [{'is_deleted': 0, 'global_id': 43526, 'PermissionFile': '47a1b076-1e38-42e3-b7fb-3909abff91be'}]}, {'is_deleted': 0, 'global_id': 22808, 'PermissionNumber': 'RU77146000-007101', 'PermissionBeginDate': '26.07.2012', 'PermissionEndDate': '31.12.2012', 'PermissionStatus': 'срок действия истек', 'PermissionFile': [{'is_deleted': 0, 'global_id': 43527, 'PermissionFile': '3df1a84d-578e-41b6-a29c-1a9bf7696a38'}]}, {'is_deleted': 0, 'global_id': 22809, 'PermissionNumber': 'RU77146000-005155', 'PermissionBeginDate': '05.07.2010', 'PermissionEndDate': '31.10.2010', 'PermissionStatus': 'срок действия истек', 'PermissionFile': [{'is_deleted': 0, 'global_id': 43528, 'PermissionFile': '957c7c1a-314a-4bf5-a10a-ee1a9806b688'}]}, {'is_deleted': 0, 'global_id': 22810, 'PermissionNumber': 'RU77146000-004099', 'PermissionBeginDate': '28.10.2009', 'PermissionEndDate': '28.02.2010', 'PermissionStatus': 'срок действия истек', 'PermissionFile': [{'is_deleted': 0, 'global_id': 43529, 'PermissionFile': '03843379-1c7b-4893-a519-a37779a1f292'}]}, {'is_deleted': 0, 'global_id': 22811, 'PermissionNumber': 'RU77146000-001694', 'PermissionBeginDate': '28.01.2008', 'PermissionEndDate': '01.08.2009', 'PermissionStatus': 'срок действия истек', 'PermissionFile': [{'is_deleted': 0, 'global_id': 43530, 'PermissionFile': 

In [10]:
ecology[ecology['start'].str.len() < 1]

,start,end,air,mean,noise,dirt,digging,capital,house_lines,clinics,roads,parks,transports,city_places,live_area,streets,trashes,factories,markets,autostations,gasstations,bad_gasstations,buildgrounds
185065,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.582339408, 55.751895783], 'type': 'Point'}"
185066,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.580817726, 55.570154208], 'type': 'Point'}"
185068,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.500756262, 55.801785039], 'type': 'Point'}"
185070,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.596979823, 55.715937227], 'type': 'Point'}"
185076,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.48556881, 55.786097706], 'type': 'Point'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185821,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.400555311, 55.643330548], 'type': 'Point'}"
185822,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.65791113, 55.637167012], 'type': 'Point'}"
185823,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.735652297, 55.776123852], 'type': 'Point'}"
185824,,27.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.564242096, 55.663143468], 'type': 'Point'}"
